In [1]:
using CSV
using DataFrames

# Ruta al archivo CSV
file_path = "C:\\Users\\leonf\\OneDrive\\Documentos\\AA_PRIMER_CUATRI\\MAAAI\\MAAA1\\Datos_Práctica_Evaluación_1.csv"

# Cargar los datos
data = CSV.read(file_path, DataFrame)

# Descripción de los datos
println("Descripción de los datos:")
println("Número de variables (columnas): ", size(data, 2))
println("Número de instancias (filas): ", size(data, 1))

# Si los datos representan individuos únicos, se puede hacer un análisis adicional
if :id in names(data) # Asumiendo que existe una columna identificadora llamada "id"
    println("Número de individuos únicos: ", length(unique(data.id)))
end

# Si es un problema de clasificación, identificar las clases de salida
# Asumiendo que la variable de salida se llama "target" o similar
if :target in names(data)
    println("Número de clases de salida: ", length(unique(data.target)))
end


Descripción de los datos:
Número de variables (columnas): 563
Número de instancias (filas): 10299
